In [1]:
# tags to identify this iteration when submitted
# example: codex_tags = {'env': 'dev', 'region': 'USA', 'product_category': 'A'}

codex_tags = {
}

from codex_widget_factory import utils
results_json=[]

In [ ]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
step_1 = """
# Below codestring is used to perform detailed analysis on quantity of sales done over time. 

import plotly.express as px
import pandas as pd
import json
from io import StringIO
import plotly.io as io


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def getGraph(dframe, filters):
    logger.info(
        "Preparing bar graph json to understand basic_expenditure across all countries")
    column_names = ['x48', 'x38', 'x37', 'x36', 'x33', 'x8', 'x29']
    #print(dframe['country'])
    data_good = dframe[dframe["data_quality"] == 1]
    data_good['basic_expenditure']= data_good[column_names].sum(axis=1)
    for item in filters:
        if 'All' in filters[item]:
            continue
        elif isinstance(filters[item], list):
            data_good = data_good[data_good[item].isin(filters[item])]
        else:
            data_good = data_good[data_good[item] == filters[item]]
    fig = px.bar(data_good, x='x48', y='country', color='basic_expenditure')
    #fig.show()
    logger.info(
        "Successfully prepared bar graph json to understand basic_expenditure across all countries")
    return io.to_json(fig)


selected_filters = {"country": ['India','Denmark']}
dframe = fiile_read("cost-of-living_v2_I0869.csv")
#dframe = dframe.groupby(['country'])
dynamic_outputs = getGraph(dframe, selected_filters)#here11

"""
# #END CUSTOM CODE

In [2]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
step_2_Filters = """
## Below codestring is used to create filters to show different regions data in  product historic screen.

import pandas as pd
import json
from io import StringIO
from itertools import chain


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(data_good)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_response_filters(current_filter_params, df, default_values_selected, all_filters, multi_select_filters, extra_filters={}):
    logger.info("Preparing filter dictionary")
    # Usage
    # -----
    # >>> filter_df = pd.DataFrame(columns=[....])    # Optional operation
    # >>> filter_df = final_ADS.groupby(......)       # Optional operation
    # >>> default_values_selected = {}    # The default value to be selected for a filter, provide filter_name, filter_values
    # >>> all_option_filters = []         # Filters with an All option
    # >>> multi_select_filters = []       # Filters with an multi_select option
    # >>> more_filters = {}               # Extra filters, provide filter_names, filter_options
    # >>> final_dict_out = get_response_filters(current_filter_params, filter_df, default_values_selected, all_option_filters, multi_select_filters, more_filters)
    # >>> dynamic_outputs = json.dumps(final_dict_out)
    # Returns
    # -------
    # A dict object containing the filters JSON structure

    filters = list(df.columns)
    default_values_possible = {}
    for item in filters:
        default_possible = list(df[item].unique())
        if item in all_filters:
            default_possible = list(chain(['All'], default_possible))
        default_values_possible[item] = default_possible
    if extra_filters:
        filters.extend(list(extra_filters.keys()))
        default_values_possible.update(extra_filters)
    if current_filter_params:
        selected_filters = current_filter_params["selected"]
        # current_filter = current_filter_params["current_filter"]
        # current_index = filters.index(current_filter)
        select_df = df.copy()
    final_dict = {}
    iter_value = 0
    data_values = []
    default_values = {}
    for item in filters:
        filter_dict = {}
        filter_dict["widget_filter_index"] = int(iter_value)
        filter_dict["widget_filter_function"] = False
        filter_dict["widget_filter_function_parameter"] = False
        filter_dict["widget_filter_hierarchy_key"] = False
        filter_dict["widget_filter_isall"] = True if item in all_filters else False
        filter_dict["widget_filter_multiselect"] = True if item in multi_select_filters else False
        filter_dict["widget_tag_key"] = str(item)
        filter_dict["widget_tag_label"] = str(item)
        filter_dict["widget_tag_input_type"] = "select",
        filter_dict["widget_filter_dynamic"] = True
        if current_filter_params:
            if item in df.columns:
                possible_values = list(select_df[item].unique())
                item_default_value = selected_filters[item]
                if item in all_filters:
                    possible_values = list(chain(['All'], possible_values))
                if item in multi_select_filters:
                    for value in selected_filters[item]:
                        if value not in possible_values:
                            if possible_values[0] == "All":
                                item_default_value = possible_values
                            else:
                                item_default_value = [possible_values[0]]
                else:
                    if selected_filters[item] not in possible_values:
                        item_default_value = possible_values[0]
                filter_dict["widget_tag_value"] = possible_values
                if item in multi_select_filters:
                    if 'All' not in item_default_value and selected_filters[item]:
                        select_df = select_df[select_df[item].isin(
                            item_default_value)]
                else:
                    if selected_filters[item] != 'All':
                        select_df = select_df[select_df[item]
                                              == item_default_value]
            else:
                filter_dict["widget_tag_value"] = extra_filters[item]
        else:
            filter_dict["widget_tag_value"] = default_values_possible[item]
            item_default_value = default_values_selected[item]
        data_values.append(filter_dict)
        default_values[item] = item_default_value
        iter_value = iter_value + 1
    final_dict["dataValues"] = data_values
    final_dict["defaultValues"] = default_values
    logger.info("Successfully prepared filter dictionary")
    return final_dict


def prepare_filter_json():
    logger.info(f"Preparing json for Filters in Historical Screen")
    # Preapre Filter json for Region in the Historical View Screen.
    dframe = fiile_read("cost-of-living_v2_I0869")
    dframe = dframe.groupby(['country']).sum().reset_index()
    filter_dframe = dframe[['country']]
    default_values_selected = {"country": 'India'}
    all_filters = []
    multi_select_filters = []
    current_filter_params = {"selected": default_values_selected}
    final_dict_out = get_response_filters(
        current_filter_params, filter_dframe, default_values_selected, all_filters, multi_select_filters)
    logger.info(f"Successful prepared json for Filters in Historical Screen")
    return json.dumps(final_dict_out)


dynamic_outputs = prepare_filter_json()
"""
#END CUSTOM CODE

In [3]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
step_2_Table = """
# Below codestring is used to display the grid table that consists of historic sales done over time on each product.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_filter_table(dframe, selected_filters):
    logger.info("Applying screen filters on the grid table dframe.")
    select_df = dframe.copy()
    for item in list(selected_filters):
        if isinstance(selected_filters[item], list):
            if 'All' not in selected_filters[item] and selected_filters[item]:
                select_df = select_df[select_df[item].isin(
                    selected_filters[item])]
        else:
            if selected_filters[item] != 'All':
                select_df = select_df[select_df[item]
                                      == selected_filters[item]]
    logger.info("Successfully applied screen filters on the grid table dframe.")
    return select_df


def generate_dynamic_table(dframe, name='Sales', grid_options={"tableSize": "small", "tableMaxHeight": "80vh", "quickSearch":True}, group_headers=[], grid="auto"):
    logger.info("Generate dynamic Grid table json from dframe")
    table_dict = {}
    table_props = {}
    table_dict.update({"grid": grid, "type": "tabularForm",
                      "noGutterBottom": True, 'name': name})
    values_dict = dframe.dropna(axis=1).to_dict("records")
    table_dict.update({"value": values_dict})
    col_def_list = []
    for col in list(dframe.columns):
        col_def_dict = {}
        col_def_dict.update({"headerName": col, "field": col})
        col_def_list.append(col_def_dict)
    table_props["groupHeaders"] = group_headers
    table_props["coldef"] = col_def_list
    table_props["gridOptions"] = grid_options
    table_dict.update({"tableprops": table_props})
    logger.info("Successfully generated dynamic Grid table json from dframe")
    return table_dict


def build_grid_table_json():
    logger.info("Preparing grid table json for Historical Screen")
    form_config = {}
    dframe = fiile_read("cost-of-living_v2_I0869")
    selected_filters = {"country": 'India'}
    dframe = get_filter_table(dframe, selected_filters)
    form_config['fields'] = [generate_dynamic_table(dframe)]
    grid_table_json = {}
    grid_table_json['form_config'] = form_config
    logger.info("Successfully prepared grid table json for Historical Screen")
    return grid_table_json


grid_table_json = build_grid_table_json()
dynamic_outputs = json.dumps(grid_table_json)

"""
#END CUSTOM CODE done till here

In [9]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
step_3_Table = """
# Below codestring is used to display the grid table that consists of historic sales done over time on each product.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def file_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869' fiile_read("cost-of-living_v2_I0869")
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x8.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        input_df = input_df[input_df.x48.notnull()]
        input_df = input_df[input_df.x37.notnull()]
        input_df = input_df[input_df.x36.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        #print(data_good)
        return(data_good)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_filter_table(dframe, selected_filters):
    logger.info("Applying screen filters on the grid table dframe.")
    select_df = dframe.copy()
    for item in list(selected_filters):
        if isinstance(selected_filters[item], list):
            if 'All' not in selected_filters[item] and selected_filters[item]:
                select_df = select_df[select_df[item].isin(
                    selected_filters[item])]
        else:
            if selected_filters[item] != 'All':
                select_df = select_df[select_df[item]
                                      == selected_filters[item]]
    logger.info("Successfully applied screen filters on the grid table dframe.")
    return select_df


def generate_dynamic_table(dframe, name='Sales', grid_options={"tableSize": "small", "tableMaxHeight": "80vh", "quickSearch":True}, group_headers=[], grid="auto"):
    logger.info("Generate dynamic Grid table json from dframe")
    table_dict = {}
    table_props = {}
    table_dict.update({"grid": grid, "type": "tabularForm",
                      "noGutterBottom": True, 'name': name})
    values_dict = dframe.dropna(axis=1).to_dict("records")
    table_dict.update({"value": values_dict})
    col_def_list = []
    for col in list(dframe.columns):
        col_def_dict = {}
        col_def_dict.update({"headerName": col, "field": col})
        col_def_list.append(col_def_dict)
    table_props["groupHeaders"] = group_headers
    table_props["coldef"] = col_def_list
    table_props["gridOptions"] = grid_options
    table_dict.update({"tableprops": table_props})
    logger.info("Successfully generated dynamic Grid table json from dframe")
    return table_dict


def build_grid_table_json():
    logger.info("Preparing grid table json for Historical Screen")
    form_config = {}
    dframe = file_read("cost-of-living_v2_I0869.csv")
    #print(dframe)
    selected_filters = {"country": 'India'}
    #print(dframe)
    dframe1=dframe[['country','x48', 'x37', 'x36', 'x8']] #--here281222 --5 columns
    print(dframe1)
    dframe = get_filter_table(dframe, selected_filters)
    form_config['fields'] = [generate_dynamic_table(dframe)]
    grid_table_json = {}
    grid_table_json['form_config'] = form_config
    logger.info("Successfully prepared grid table json for Historical Screen")
    return grid_table_json


grid_table_json = build_grid_table_json()
dynamic_outputs = json.dumps(grid_table_json)

# """
# #END CUSTOM CODE done till here

          country      x48   x37     x36    x8
0     South Korea   742.54  0.16  182.13  0.79
1           China  1091.93  0.03   66.00  0.33
2           China   533.28  0.02   59.65  0.33
3           India   522.40  0.01   43.57  0.19
4           India   229.84  0.01   58.07  0.19
...           ...      ...   ...     ...   ...
4111       Canada   925.51  0.22  149.83  1.58
4205       Norway   719.26  0.14  178.99  2.93
4322     Portugal   882.51  0.13  101.86  1.44
4404       Brazil   306.55  0.23   66.94  0.71
4494       Russia   473.14  0.04  133.70  0.50

[894 rows x 5 columns]


In [21]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
#step_4 = """
# Below codestring is used to perform detailed analysis on quantity of sales done over time. 

import plotly.express as px
import pandas as pd
import json
from io import StringIO
import plotly.io as io


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def getGraph(dframe, filters):
    logger.info(
        "Preparing bar graph json to understand basic_expenditure across all countries")
    column_names = ['x48', 'x38', 'x37', 'x36', 'x33', 'x8', 'x29']
    #print(dframe['country'])
    data_good = dframe[dframe["data_quality"] == 1]
    data_good['basic_expenditure']= data_good[column_names].sum(axis=1)
    for item in filters:
        if 'All' in filters[item]:
            continue
        elif isinstance(filters[item], list):
            data_good = data_good[data_good[item].isin(filters[item])]
        else:
            data_good = data_good[data_good[item] == filters[item]]
    #fig = px.bar(data_good, x='basic_expenditure', y='country') #, color='city'
    fig = px.scatter(data_good, y='basic_expenditure', x='country')
    #print(data_good[['basic_expenditure','country']].where(data_good['country'] == 'Denmark')) #--here3
    data_good1 = data_good[data_good['country'] == 'Denmark']
    print(data_good1)
    fig.show()
    logger.info(
        "Successfully prepared bar graph json to understand basic_expenditure across all countries")
    return io.to_json(fig)


selected_filters = {"country": ['India','Denmark']}
dframe = fiile_read("cost-of-living_v2_I0869.csv")
#dframe = dframe.groupby(['country'])
dynamic_outputs = getGraph(dframe, selected_filters)#here11

#"""
# #END CUSTOM CODE

            city  country     x1      x2     x3    x4    x5    x6    x7    x8  \
396   Copenhagen  Denmark  19.85  113.40  12.05  7.09  7.05  5.99  3.25  2.67   
1494      Aarhus  Denmark  21.26   92.14  12.47  7.80  5.67  5.61  3.37  2.84   
1929      Odense  Denmark  21.26  113.40  11.98  6.80  6.38  5.83  3.70  3.26   
2258     Aalborg  Denmark  18.43   92.14  11.34  7.09  6.02  4.81  3.69  2.61   
2926       Vejle  Denmark  17.01   85.05  12.62  7.44  5.67  6.02  4.96  3.01   

      ...      x48      x49      x50      x51      x52      x53      x54  \
396   ...  1765.18  1309.91  2669.65  2014.90  7808.48  5450.68  3670.25   
1494  ...  1064.15   670.59  2036.07  1417.51  6176.31  3307.53  3458.74   
1929  ...   885.24   666.23  1701.02  1319.86  3173.44  2338.90  2895.05   
2258  ...   757.36   543.38  1273.99  1022.19  4388.73  3349.10  3448.13   
2926  ...   726.48   661.51  1417.51  1116.29  3071.28  2268.02  3503.62   

       x55  data_quality  basic_expenditure  
396   1.69

c:\Users\SampreetGupta\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
step_4_a ="""
import numpy as np
import pandas as pd
df = pd.DataFrame(np.random.randint(0,100,size=(100, 1)), columns=list('X'))
df = df.assign(Y = lambda x: (x['X']**2))
fig = px.scatter(df, y='Y', x='X')
fig.show()
#print(df)
#"""

     X     Y
0   89  7921
1   26   676
2   87  7569
3   57  3249
4   99  9801
..  ..   ...
95  46  2116
96  38  1444
97   1     1
98  77  5929
99  94  8836

[100 rows x 2 columns]


In [53]:
#step_5 ="""

#"""import plotly.graph_objects as go

import pandas as pd
import numpy as np
import plotly.graph_objects as go
# load dataset
df = pd.DataFrame(np.random.randint(0,100,size=(100, 1)), columns=list('X'))
df = df.assign(Y = lambda x: (x['X']**2))
#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/volcano.csv")
#print(df)
# create figure
fig = go.Figure()

# Add surface trace
fig.add_trace(go.Surface(  x=df.X,y=df.Y, colorscale="Viridis"))

# # Update plot sizing
# fig.update_layout(
#     width=800,
#     height=900,
#     autosize=False,
#     margin=dict(t=0, b=0, l=0, r=0),
#     template="plotly_white",
# )

# Update 3D scene options
fig.update_scenes(
    aspectratio=dict(x=1, y=1, z=0.7),
    aspectmode="manual"
)

# Add dropdown
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["type", "bar"],
                    label="bar",
                    method="restyle"
                ),
                dict(
                    args=["type", "box"],
                    label="box",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="Trace type:", showarrow=False,
        x=0, y=1.085, yref="paper", align="left")
    ]
)

fig.show()

In [4]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
historicScreenActionGen = """
# Below codestring is used to create action generator that triggers action handler  to display breadcrumbs 
# to show total units of product sale over time.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def get_action_generator():
    logger.info(f"Preparing action generator json for Historical screen")
    action_generator = {
        "actions": [{
            "action_type": "get_screen_breadcrumbs",
            "component_type": "text_list",
            "params": {
                "fetch_on_load": True
            },
            "position": {
                "portal": "screen_top_left"
            }
        }]
    }

    logger.info(
        f"Successfully prepared action generator json for Historical screen")
    return action_generator


res = get_action_generator()
dynamic_outputs = json.dumps(res)

"""
#END CUSTOM CODE

In [5]:
#put your output in this response param for connecting to downstream widgets

historicScreenActionHandler = """
# Below codestring is used to generate json for breadcrumbs to display total number units of sales done over time.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(data_good)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_quantity_breadcrumbs():
    logger.info(
        f"Preparing action handler json for number of cities in historical screen")
    dframe = fiile_read("cost-of-living_v2_I0869")
    total_cities = dframe.city.nunique()
    actions = {
        "list": [
            {
                "text": "number of cities: ",
            },
            {
                "text": str(total_cities),
                "color": "contrast",
                "style": {
                    "fontWeight": 600,
                }
            }
        ]
    }
    logger.info(
        f"Successfully prepared action handler json for order quantity in historical screen")
    return actions


# action_type = "get_screen_breadcrumbs"


if action_type == "get_screen_breadcrumbs":
    res = get_quantity_breadcrumbs()

dynamic_outputs = json.dumps(res)

"""

In [6]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
top_10_costly_countries = """
#productReturn_quantity 
# Below codestring is used to display the nuumber of product unit returned in a year.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(input_df)
        return(data_good)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


# def get_productReturnQuantity():
#     logger.info("Calculating Quantity of product returned")
#     dframe = fiile_read("cost-of-living_v2_I0869") #*test#
#     productReturnQuantity = dframe['ReturnQuantity'].sum()
#     metric = {}
#     metric['title'] = "Quantity of Products Returned"
#     metric['value'] = str(int(productReturnQuantity)) + ' units'
#     logger.info("Successfully calculated Quantity of product returned")
#     return metric

def get_count_of_countries_top_10_percentile_cost_wise():
    logger.info("Calculating Quantity of product returned")
    dframe = fiile_read("cost-of-living_v2_I0869") #*test#
    dframe['Percentile_Rank'] = dframe.x48.rank(pct = True)
    filtered_df= dframe[dframe['Percentile_Rank']> 0.9]
    cnt =filtered_df.country.nunique()
    metric = {}
    metric['title'] = "top_10_percentile_countries_cost_wise"
    metric['value'] = str(int(cnt)) + ' countries'
    logger.info("Successfully calculated Quantity of product returned")
    return metric

kpi_json = get_productReturnQuantity()
dynamic_outputs = json.dumps(kpi_json)
"""
#END CUSTOM CODE

In [7]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
productReturn_UniqueProduct = """
# Below codestring is used to display the nuumber of uniue product types returned in a year.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(input_df)
        return(data_good)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


# def get_uniqueProductReturned():
#     logger.info("Calculating number of unique products returned")
#     dframe = fiile_read("cost-of-living_v2_I0869")
#     uniqueProductQuantity = len(dframe['ProductName'].unique())
#     metric = {}
#     metric['title'] = "No of Unique Products Returned"
#     metric['value'] = str(int(uniqueProductQuantity)) + ' unique Products'
#     logger.info("Successfully calculated number of unique products returned")
#     return metric

def get_count_of_countries_top_10_percentile_income_wise():
    #logger.info("Calculating Quantity of product returned")
    logger.info("count_of_countries_top_10_percentile_income_wise")
    dframe = fiile_read("cost-of-living_v2_I0869") #*test#
    dframe['Percentile_Rank'] = dframe.x54.rank(pct = True)
    filtered_df= dframe[dframe['Percentile_Rank']> 0.9]
    cnt =filtered_df.country.nunique()
    metric = {}
    metric['title'] = "count_of_countries_top_10_percentile_income_wise"
    metric['value'] = str(int(cnt)) + ' countries'
    logger.info("Successfully calculated Quantity of product returned")
    return metric


kpi_json = get_uniqueProductReturned()
dynamic_outputs = json.dumps(kpi_json)
"""
#END CUSTOM CODE

In [8]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
productReturns_costOfProductReturned = """
# Below codestring is used to display the total cost of product that was returned in a year.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def read_dataset(filename):
    # Read dataset from the github.
    logger.info(f"Read dataset file: {filename}")
    try:
        dataset_url = f"https://raw.githubusercontent.com/saipraneeth4/codx_dataset/main/dataset/{filename}"
        dframe = pd.read_csv(dataset_url)
        return dframe
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {dataset_url}"
                    f"Error Info is  {error_msg}")


# def get_costOfProductReturned():
#     logger.info("Calculating Totalcost of products returned")
#     dframe = fiile_read("cost-of-living_v2_I0869")
#     costOfProdRetuned = dframe['ProductPrice'].sum()
#     metric = {}
#     metric['title'] = "No of Unique Products Returned"
#     metric['value'] = str(int(costOfProdRetuned)) + ' $'
#     logger.info("Successfully calculated Totalcost of products returned")
#     return metric

def top_ten_percentile_in_savings():
    logger.info("net_cost_of_living_baisc_ammenities")
    dframe = fiile_read("cost-of-living_v2_I0869")
    dframe1=dframe.groupby(['country'])['basic_expenditure', 'x54','country'].mean()
    dframe1['savings'] = dframe1['x54'] - dframe1['basic_expenditure']
    dframe1['percentage_savings'] = dframe1['savings'] * 100 / dframe1['x54']
    dframe1['Percentile_Rank_savings'] = dframe1.savings.rank(pct = True)
    filtered_df= dframe1[dframe1['Percentile_Rank_savings']> 0.9]
    ##print(filtered_df.head(10))
    cnt =len(filtered_df)
    ##print(dframe1.sort_values(by=['savings'], ascending=False))
    #print(cnt)
    metric = {}
    metric['title'] = "No of countries in top_ten_percentile_in_savings"
    metric['value'] = cnt
    logger.info("Successfully calculated the No of countries in top_ten_percentile_in_savings")
    return metric
    


kpi_json = top_ten_percentile_in_savings()
dynamic_outputs = json.dumps(kpi_json)
"""
#END CUSTOM CODE

In [10]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
oderDetailProduct = """
# Below codestring is used to perform detailed analysis of Product stock available over time. 

import plotly.express as px
import pandas as pd
import json
from io import StringIO
import plotly.io as io


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def getGraph(dframe, filters):
    logger.info(
        "Preparing bar graph json to understand net income across all countries")
    for item in filters:
        if 'All' in filters[item]:
            continue
        elif isinstance(filters[item], list):
            dframe = dframe[dframe[item].isin(filters[item])]
        else:
            dframe = dframe[dframe[item] == filters[item]]
    fig = px.bar(dframe, x='city', y='x54', color='x48')
    # fig.show()
    logger.info(
        "Successfully prepared bar graph json to understand net income across all countries")
    return io.to_json(fig)


selected_filters = {"Country": 'India'}
dframe = fiile_read("cost-of-living_v2_I0869")
dynamic_outputs = getGraph(dframe, selected_filters)
"""
#END CUSTOM CODE

In [11]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
productReturns_gridTable = """
# Below codestring is used to view the product returns details.

import pandas as pd
import json
from io import StringIO


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()


def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def generate_dynamic_table(dframe, name='Sales', grid_options={"tableSize": "small", "tableMaxHeight": "80vh", "quickSearch":True}, group_headers=[], grid="auto"):
    logger.info("Generate dynamic Grid table json from dframe")
    table_dict = {}
    table_props = {}
    table_dict.update({"grid": grid, "type": "tabularForm",
                      "noGutterBottom": True, 'name': name})
    values_dict = dframe.dropna(axis=1).to_dict("records")
    table_dict.update({"value": values_dict})
    col_def_list = []
    for col in list(dframe.columns):
        col_def_dict = {}
        col_def_dict.update({"headerName": col, "field": col})
        col_def_list.append(col_def_dict)
    table_props["groupHeaders"] = group_headers
    table_props["coldef"] = col_def_list
    table_props["gridOptions"] = grid_options
    table_dict.update({"tableprops": table_props})
    logger.info("Successfully generated dynamic Grid table json from dframe")
    return table_dict


def build_grid_table_json():
    logger.info("Preparing grid table json for Product Returns Screen")
    form_config = {}
    dframe = fiile_read("cost-of-living_v2_I0869")
    form_config['fields'] = [generate_dynamic_table(dframe)]
    grid_table_json = {}
    grid_table_json['form_config'] = form_config
    logger.info("Successfully prepared grid table json for Product Returns Screen")
    return grid_table_json


grid_table_json = build_grid_table_json()
dynamic_outputs = json.dumps(grid_table_json)
"""
#END CUSTOM CODE

In [12]:
#BEGIN CUSTOM CODE BELOW...

#put your output in this response param for connecting to downstream widgets
orderDetailFilter = """
## Below codestring is used to create filters to show different regions data in  Order Detail Analysis screen.

import pandas as pd
import json
from io import StringIO
from itertools import chain


def getLogger():
    import logging
    logging.basicConfig(filename="UIACLogger.log",
                        format='%(asctime)s %(message)s',
                        filemode='a')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    return logger


logger = getLogger()

def fiile_read(blob_name):
    #reading the file from azure blob storage container
    logger.info(f"Read dataset file: {blob_name}")
    try:
        from azure.storage.blob import BlockBlobService
        sas_token = '?sv=2021-04-10&st=2022-12-22T08%3A12%3A47Z&se=2023-12-30T08%3A12%3A00Z&sr=c&sp=racwl&sig=fMeYkXsCvwK%2F0qVrCmj2j3NiMricQjOPWOkAEXekIPA%3D'
        account_name = 'willbedeletedsoon'
        container_name = 'codx-pede-s02'
        #blob_name = 'cost-of-living_v2_I0869'
        def get_data_from_blob(sas_token, account_name, container_name, blob_name):
            block_blob_service = BlockBlobService(account_name=account_name, sas_token= sas_token)
            from_blob = block_blob_service.get_blob_to_text(container_name = container_name, blob_name=blob_name)
            return pd.read_csv(StringIO(from_blob.content))
        input_df=get_data_from_blob(sas_token, account_name, container_name, blob_name)
        #filtering out empty columns
        input_df = input_df[input_df.x2.notnull()]
        input_df = input_df[input_df.x23.notnull()]
        input_df = input_df[input_df.x33.notnull()]
        input_df = input_df[input_df.x54.notnull()]
        data_good = input_df[input_df["data_quality"] == 1]
        
        ##print(data_good)
        return(input_df)
    except Exception as error_msg:
        logger.info(f"Exception occured while reading the dataset: {blob_name}"
                    f"Error Info is  {error_msg}")


def get_response_filters(current_filter_params, df, default_values_selected, all_filters, multi_select_filters, extra_filters={}):
    logger.info("Preparing filter dictionary")
    # Usage
    # -----
    # >>> filter_df = pd.DataFrame(columns=[....])    # Optional operation
    # >>> filter_df = final_ADS.groupby(......)       # Optional operation
    # >>> default_values_selected = {}    # The default value to be selected for a filter, provide filter_name, filter_values
    # >>> all_option_filters = []         # Filters with an All option
    # >>> multi_select_filters = []       # Filters with an multi_select option
    # >>> more_filters = {}               # Extra filters, provide filter_names, filter_options
    # >>> final_dict_out = get_response_filters(current_filter_params, filter_df, default_values_selected, all_option_filters, multi_select_filters, more_filters)
    # >>> dynamic_outputs = json.dumps(final_dict_out)
    # Returns
    # -------
    # A dict object containing the filters JSON structure

    filters = list(df.columns)
    default_values_possible = {}
    for item in filters:
        default_possible = list(df[item].unique())
        if item in all_filters:
            default_possible = list(chain(['All'], default_possible))
        default_values_possible[item] = default_possible
    if extra_filters:
        filters.extend(list(extra_filters.keys()))
        default_values_possible.update(extra_filters)
    if current_filter_params:
        selected_filters = current_filter_params["selected"]
        # current_filter = current_filter_params["current_filter"]
        # current_index = filters.index(current_filter)
        select_df = df.copy()
    final_dict = {}
    iter_value = 0
    data_values = []
    default_values = {}
    for item in filters:
        filter_dict = {}
        filter_dict["widget_filter_index"] = int(iter_value)
        filter_dict["widget_filter_function"] = False
        filter_dict["widget_filter_function_parameter"] = False
        filter_dict["widget_filter_hierarchy_key"] = False
        filter_dict["widget_filter_isall"] = True if item in all_filters else False
        filter_dict["widget_filter_multiselect"] = True if item in multi_select_filters else False
        filter_dict["widget_tag_key"] = str(item)
        filter_dict["widget_tag_label"] = str(item)
        filter_dict["widget_tag_input_type"] = "select",
        filter_dict["widget_filter_dynamic"] = True
        if current_filter_params:
            if item in df.columns:
                possible_values = list(select_df[item].unique())
                item_default_value = selected_filters[item]
                if item in all_filters:
                    possible_values = list(chain(['All'], possible_values))
                if item in multi_select_filters:
                    for value in selected_filters[item]:
                        if value not in possible_values:
                            if possible_values[0] == "All":
                                item_default_value = possible_values
                            else:
                                item_default_value = [possible_values[0]]
                else:
                    if selected_filters[item] not in possible_values:
                        item_default_value = possible_values[0]
                filter_dict["widget_tag_value"] = possible_values
                if item in multi_select_filters:
                    if 'All' not in item_default_value and selected_filters[item]:
                        select_df = select_df[select_df[item].isin(
                            item_default_value)]
                else:
                    if selected_filters[item] != 'All':
                        select_df = select_df[select_df[item]
                                              == item_default_value]
            else:
                filter_dict["widget_tag_value"] = extra_filters[item]
        else:
            filter_dict["widget_tag_value"] = default_values_possible[item]
            item_default_value = default_values_selected[item]
        data_values.append(filter_dict)
        default_values[item] = item_default_value
        iter_value = iter_value + 1
    final_dict["dataValues"] = data_values
    final_dict["defaultValues"] = default_values
    logger.info("Successfully prepared filter dictionary")
    return final_dict


def prepare_filter_json():
    logger.info(f"Preparing json for Filters in Order Detail Screen")
    # Preapre Filter json for Region in the Order Detail View Screen.
    dframe = fiile_read("cost-of-living_v2_I0869")
    dframe = dframe.groupby(['country']).sum().reset_index()
    filter_dframe = dframe[['country']]
    default_values_selected = {"country": 'India'}
    all_filters = []
    multi_select_filters = []
    current_filter_params = {"selected": default_values_selected}
    final_dict_out = get_response_filters(
        current_filter_params, filter_dframe, default_values_selected, all_filters, multi_select_filters)
    logger.info(f"Successful prepared json for Filters in Order Detail Screen")
    return json.dumps(final_dict_out)


dynamic_outputs = prepare_filter_json()

"""
#END CUSTOM CODE

In [13]:
# dynamic_metrics = {
#     'Temp of countries more than 35': temp_of_more_than_35,
#     'Total Unique Product Quantity Returned': productReturn_UniqueProduct,
#     'Total Cost of Product Returned': productReturns_costOfProductReturned,
# }




# dynamic_filter = {
#     'HistoricScreenFilters': HistoricScreenFilters,
#     'orderDetailFilter': orderDetailFilter,
# }
# dynamic_actions = {
#     "breadcrumb": {
#         "default": None,
#         "action_generator": historicScreenActionGen,
#         "action_handler": historicScreenActionHandler
#     },
# }

# results_json.append({
# #     'type': 'Dynamic Plot',
# #     'name': 'Dynamic Plot',
# #     'component': 'Dynamic Plot',
# #     'dynamic_visual_results': dynamic_result,
# #     'dynamic_code_filters': dynamic_filter,
# #     'dynamic_metrics_results': dynamic_metrics,
# #     'actions': dynamic_actions

# # })
dynamic_result = {
    'Product Details Historic View': historicScreenGridTable,
 }
results_json.append({
    'type': 'Dynamic Plot',
    'name': 'Dynamic Plot',
    'component': 'Dynamic Plot',
    'dynamic_visual_results': dynamic_result,
    # 'dynamic_code_filters': dynamic_filter,
    # 'dynamic_metrics_results': dynamic_metrics,
    # 'actions': dynamic_actions

})




In [17]:
currentNotebook = 'Downloads\depe1.ipynb'

!jupyter nbconvert --to script {currentNotebook} 

Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "C:\Users\SampreetGupta\AppData\Local\Programs\Python\Python310\lib\site.py", line 617, in <module>
    main()
  File "C:\Users\SampreetGupta\AppData\Local\Programs\Python\Python310\lib\site.py", line 604, in main
    known_paths = addsitepackages(known_paths)
  File "C:\Users\SampreetGupta\AppData\Local\Programs\Python\Python310\lib\site.py", line 387, in addsitepackages
    addsitedir(sitedir, known_paths)
  File "C:\Users\SampreetGupta\AppData\Local\Programs\Python\Python310\lib\site.py", line 226, in addsitedir
    addpackage(sitedir, name, known_paths)
  File "C:\Users\SampreetGupta\AppData\Local\Programs\Python\Python310\lib\site.py", line 175, in addpackage
    f = io.TextIOWrapper(io.open_code(fullname), encoding="locale")
  File "C:\Users\SampreetGupta\AppData\Local\Programs\Python\Python310\lib\encodings\__init__.py", line 99, in s

In [18]:

utils.submit_config_params(url='https://codex-api-stage.azurewebsites.net/codex-api/projects/upload-config-params/XbpsYR9JylEWw7cfZo3YLw', nb_name=currentNotebook, results=results_json, codex_tags=codex_tags, args={})

FileNotFoundError: [Errno 2] No such file or directory: 'Downloads\\depe1.py'